In [1]:
#라이브러리 갖고오기

import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm #Analysis
from sklearn.preprocessing import StandardScaler #Analysis
from scipy import stats #Analysis

In [2]:
#파일 경로 지정

data_dir = "~/aiffel/kaggle_kakr_housing/data"
train_data_path = join(data_dir, 'train.csv')
sub_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

print(train_data_path)
print(sub_data_path)

~/aiffel/kaggle_kakr_housing/data/train.csv
~/aiffel/kaggle_kakr_housing/data/test.csv


In [3]:
#데이터 호출
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(sub_data_path)

In [4]:
# 데이터 shape확인
print(df_train.shape)
print(df_test.shape)

(15035, 21)
(6468, 20)


In [5]:
# date의 경우 yyyymmdd+T000000의 형태로 T000000은 의미 없는 데이터므로 이후에 삭제해주는 것이 좋을듯.
df_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [6]:
# 결측치 확인
# 확인상 데이터 결측치는 존재하지 않음
df_train.isnull().sum()
df_test.isnull().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [9]:
train = pd.read_csv(train_data_path)
test = pd.read_csv(sub_data_path)

In [10]:
# 타켓, id 데이터 삭제 for trianing
# date change
# train['year'] = train['date'].apply(lambda i : i[:4]).astype(int)
# train['month'] = train['date'].apply(lambda i : i[4:6]).astype(int)
# train['day'] = train['date'].apply(lambda i : i[6:8]).astype(int)
train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)


y = train['price']
del train['price']
del train['id']

# test의 id 컬럼 삭제
# test['year'] = test['date'].apply(lambda i : i[:4]).astype(int)
# test['month'] = test['date'].apply(lambda i : i[4:6]).astype(int)
# test['day'] = test['date'].apply(lambda i : i[6:8]).astype(int)
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)

del test['id']

# y log change
y = np.log1p(y)

In [12]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [14]:
random_state=2020   

In [16]:
from sklearn.model_selection import RandomizedSearchCV

def my_RandomSearch(model, train, y, param, n_iter= 5, verbose=1, n_jobs=5):

    # 1. GridSearchCV 모델로 `model`을 초기화합니다.
    random_model = RandomizedSearchCV(model, param_distributions=param, \
                        scoring='neg_mean_squared_error', \
                              n_iter =  n_iter, \
                        cv=10, verbose=1, n_jobs=-1)

    # 2. 모델을 fitting 합니다.
    random_model.fit(train, y)

    # 3. params, score에 각 조합에 대한 결과를 저장합니다.
    params = random_model.cv_results_['params']
    score = random_model.cv_results_['mean_test_score']

    # 4. 데이터 프레임을 생성
    results = pd.DataFrame(params)
    results["score"] = score

    # 5. RMSLE 값을 추가한 후 점수가 높은 순서로 정렬한 `results`를 반환합니다.
    results['RMSLE'] = np.sqrt(-1 * results['score'])

    results = results.sort_values("RMSLE")
    return results

In [17]:
#try after getting score which is lower 110000
param = {
    'n_estimators': [int(x) for x in range(500,600,2)],
    'max_depth': [int(x) for x in range(0,15)],
}
n_iter = 50

model = LGBMRegressor(random_state=random_state)
my_RandomSearch(model, train, y, param, n_iter, verbose=1)

#range(150,300,2) 진행결과 estimator : 288, max_depth : 10 가 rmsle 0.161052
#range(300,400,2) 진행결과 estimator : 380, max_depth : 8 가 rmsle 0.160893
#range(400,450,2) 진행결과 estimator : 404, max_depth : 0 가 rmsle 0.160915
#range(400,450,2) 진행결과 estimator : 402, max_depth : 8 가 rmsle 0.160941
#range(400,450,2) 진행결과 estimator : 402, max_depth : 8 가 rmsle 0.160941

Fitting 10 folds for each of 50 candidates, totalling 500 fits


,n_estimators,max_depth,score,RMSLE
8,550,4,-0.025970,0.161151
30,594,10,-0.025974,0.161165
12,506,0,-0.025981,0.161185
0,540,4,-0.025986,0.161201
37,510,8,-0.026006,0.161263
9,508,8,-0.026010,0.161276
15,506,12,-0.026015,0.161292
27,514,7,-0.026016,0.161294
18,508,4,-0.026020,0.161308
21,514,12,-0.026020,0.161309


In [18]:
model = LGBMRegressor(max_depth=4, n_estimators=550, random_state=random_state)
model.fit(train, y)

prediction = model.predict(test)
prediction = np.expm1(prediction)
prediction

array([ 511124.37324754,  474758.75863868, 1388515.81487495, ...,
        473566.1654046 ,  322314.28243738,  467814.65175187])

In [19]:
submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission['price'] = prediction
submission_csv_path = 'submission_{}_RMSLE_{}.csv'.format( 'lgbm', '0.164')
submission.to_csv('submission4.csv', index = False)

In [ ]:
#try after getting score which is lower 110000
param = {
    'n_estimators': [int(x) for x in range(500,600,2)],
    'max_depth': [int(x) for x in range(0,15)],
}
n_iter = 50

model = XGBRegressor(random_state=random_state)
my_RandomSearch(model, train, y, param, n_iter, verbose=1)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(


In [ ]:
model = XGBRegressor(max_depth=4, n_estimators=550, random_state=random_state)
model.fit(train, y)

prediction = model.predict(test)
prediction = np.expm1(prediction)
prediction

In [ ]:
submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission['price'] = prediction
submission_csv_path = 'submission_{}_RMSLE_{}.csv'.format( 'lgbm', '0.164')
submission.to_csv('submission5.csv', index = False)